# Aggregating counts and rates of hospital admissions and make a table

In [1]:
import pandas as pd
import numpy as np
import functools as ft

## 0- functions and read data

In [2]:
def data_process(data, data_hosped, infection):
    #replace SMs with 5 in data and data_hosped and NaN with 0 in data_hosped 
    data = data.replace({'SM': 5})
    data_hosped = data_hosped.replace({'SM': 5, np.nan:0})
    #conver counts into int
    data[infection] = data[infection].astype(str).replace('\.0', '', regex=True).astype(int)
    data_hosped[infection] = data_hosped[infection].astype(str).replace('\.0', '', regex=True).astype(int)

    #add hosp rate
    data_hosped[infection+'_rate'] = (data_hosped[infection]/data[infection])*1000
    data_hosped[infection+'_rate'] = np.round(data_hosped[infection+'_rate'], decimals=1)

    #replace rate with SN if infection count equals 5
    data_hosped.loc[data_hosped[infection] == 5, infection+'_rate'] = 'SN'

    #calculate total using sex_female and sex_male
    total = data_hosped.iloc[int(data_hosped[data_hosped['variable']=='sex_male'].index[0])][infection] + data_hosped.iloc[int(data_hosped[data_hosped['variable']=='sex_female'].index[0])][infection]
    total = "{:,}".format(total)
    data_hosped['variable'] = data_hosped['variable'].str.replace('antibacterial_brit', 'total')
    
    #add comma to thousands of column count
    data_hosped[infection] = data_hosped[infection].apply(lambda x : "{:,}".format(x))
    #make new column with counts and rates
    data_hosped[infection+'_rate_count'] = data_hosped[infection].astype(str) + ' (' + data_hosped[infection+'_rate'].astype(str) + ')'
    
    #replace total (after making new column of counts and rates)
    data_hosped.at[int(data_hosped[data_hosped['variable']=='total'].index[0]),infection+'_rate_count'] = total

    #drop redundant columns and rename the needed column
    data_hosped = data_hosped.drop([infection, infection+'_rate'], axis=1)
    data_hosped = data_hosped.rename(columns={infection+'_rate_count': infection})
    #replace any row with no value (=0) with a dash 
    data_hosped[infection] = data_hosped[infection].replace({'0 (0.0)': '-'})
    #replace count and rate if rate contains SN
    data_hosped.loc[data_hosped[infection].str.contains('SN', regex=False), infection] = 'SN' 

    return data_hosped

In [3]:
summary_table_cases_lrti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_incdt_cases.csv')
summary_table_cases_lrti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_prevl_cases.csv')
summary_table_cases_urti_combined_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_incdt_cases.csv')
summary_table_cases_urti_combined_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_prevl_cases.csv')
summary_table_cases_uti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_incdt_cases.csv')
summary_table_cases_uti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_prevl_cases.csv')
summary_table_cases_sinusitis_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_incdt_cases.csv')
summary_table_cases_sinusitis_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_prevl_cases.csv')
summary_table_cases_otmedia_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_incdt_cases.csv')
summary_table_cases_otmedia_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_prevl_cases.csv')
summary_table_cases_ot_externa_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_incdt_cases.csv')
summary_table_cases_ot_externa_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_prevl_cases.csv')
summary_table_cases_urti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_incdt_cases.csv')
summary_table_cases_urti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_prevl_cases.csv')
summary_table_cases_cough_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_incdt_cases.csv')
summary_table_cases_cough_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_prevl_cases.csv')
summary_table_cases_cough_cold_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_incdt_cases.csv')
summary_table_cases_cough_cold_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_prevl_cases.csv')
summary_table_cases_throat_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_incdt_cases.csv')
summary_table_cases_throat_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_prevl_cases.csv')

In [4]:
summary_table_lrti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_incdt.csv')
summary_table_lrti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_prevl.csv')
summary_table_urti_combined_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_incdt.csv')
summary_table_urti_combined_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_prevl.csv')
summary_table_uti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_incdt.csv')
summary_table_uti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_prevl.csv')
summary_table_sinusitis_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_incdt.csv')
summary_table_sinusitis_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_prevl.csv')
summary_table_otmedia_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_incdt.csv')
summary_table_otmedia_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_prevl.csv')
summary_table_ot_externa_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_incdt.csv')
summary_table_ot_externa_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_prevl.csv')
summary_table_urti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_incdt.csv')
summary_table_urti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_prevl.csv')
summary_table_cough_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_incdt.csv')
summary_table_cough_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_prevl.csv')
summary_table_cough_cold_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_incdt.csv')
summary_table_cough_cold_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_prevl.csv')
summary_table_throat_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_incdt.csv')
summary_table_throat_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_prevl.csv')

In [5]:
#select relevant columns of each df
summary_table_lrti_incdt = summary_table_lrti_incdt[['variable', 'count']]
summary_table_lrti_prevl = summary_table_lrti_prevl[['variable', 'count']]
summary_table_urti_combined_incdt = summary_table_urti_combined_incdt[['variable', 'count']]
summary_table_urti_combined_prevl = summary_table_urti_combined_prevl[['variable', 'count']]
summary_table_uti_incdt = summary_table_uti_incdt[['variable', 'count']]
summary_table_uti_prevl = summary_table_uti_prevl[['variable', 'count']]
summary_table_sinusitis_incdt = summary_table_sinusitis_incdt[['variable', 'count']]
summary_table_sinusitis_prevl = summary_table_sinusitis_prevl[['variable', 'count']]
summary_table_otmedia_incdt = summary_table_otmedia_incdt[['variable', 'count']]
summary_table_otmedia_prevl = summary_table_otmedia_prevl[['variable', 'count']]
summary_table_ot_externa_incdt = summary_table_ot_externa_incdt[['variable', 'count']]
summary_table_ot_externa_prevl = summary_table_ot_externa_prevl[['variable', 'count']]
summary_table_urti_incdt = summary_table_urti_incdt[['variable', 'count']]
summary_table_urti_prevl = summary_table_urti_prevl[['variable', 'count']]
summary_table_cough_incdt = summary_table_cough_incdt[['variable', 'count']]
summary_table_cough_prevl = summary_table_cough_prevl[['variable', 'count']]
summary_table_cough_cold_incdt = summary_table_cough_cold_incdt[['variable', 'count']]
summary_table_cough_cold_prevl = summary_table_cough_cold_prevl[['variable', 'count']]
summary_table_throat_incdt = summary_table_throat_incdt[['variable', 'count']]
summary_table_throat_prevl = summary_table_throat_prevl[['variable', 'count']]

In [6]:
#merge baseline dfs
dfs_tab = [summary_table_lrti_incdt, summary_table_lrti_prevl, 
        summary_table_urti_combined_incdt, summary_table_urti_combined_prevl, 
        summary_table_uti_incdt, summary_table_uti_prevl, 
        summary_table_sinusitis_incdt, summary_table_sinusitis_prevl, 
        summary_table_otmedia_incdt, summary_table_otmedia_prevl, 
        summary_table_ot_externa_incdt, summary_table_ot_externa_prevl, 
        summary_table_urti_incdt, summary_table_urti_prevl, 
        summary_table_cough_incdt, summary_table_cough_prevl, 
        summary_table_cough_cold_incdt, summary_table_cough_cold_prevl, 
        summary_table_throat_incdt, summary_table_throat_prevl, 
        ]

#replace any column that contains ab_date with ab_binray
for i in dfs_tab:
    i.loc[i['variable'].str.contains('_ab_date'), 'variable'] = 'ab_binary'

#concat dfs and assign column names
df_concated = pd.concat([i.set_index('variable') for i in dfs_tab],axis=1, join='outer').reset_index()
df_concated.columns = ['variable', 
                        'lrti_incdt', 'lrti_prevl', 
                        'urti_combined_incdt', 'urti_combined_prevl',
                        'uti_incdt', 'uti_prevl',
                        'sinusitis_incdt', 'sinusitis_prevl',
                        'otmedia_incdt', 'otmedia_prevl',
                        'ot_externa_incdt', 'ot_externa_prevl',
                        'urti_incdt', 'urti_prevl',
                        'cough_incdt', 'cough_prevl',
                        'cough_cold_incdt', 'cough_cold_prevl',
                        'throat_incdt', 'throat_prevl'
                        ]

#custom sort baseline dfs
df_concated['variable'] = pd.Categorical(df_concated['variable'], ['age_cat_15_24', 'age_cat_25_34', 'age_cat_35_44', 'age_cat_45_54', 'age_cat_55_64', 'age_cat_65_74', 'age_cat_75_more',
                                                                   'sex_female', 'sex_male',
                                                                   'CCI_cat_high', 'CCI_cat_low', 'CCI_cat_medium', 'CCI_cat_very_high', 'CCI_cat_very_low',
                                                                   'flu_vaccine_no', 'flu_vaccine_yes',
                                                                   'bmi_cat_healthy_weight', 'bmi_cat_obese', 'bmi_cat_overweight', 'bmi_cat_underweight', 'bmi_cat_unknown',
                                                                   'region_east', 'region_east_midlands', 'region_london', 'region_north_east', 'region_north_west', 'region_south_east', 'region_south_west', 'region_west_midlands', 'region_yorkshire',
                                                                   'imd_affluent', 'imd_medium', 'imd_unaffluent', 'imd_unknown', 'imd_very_affluent', 'imd_very_unaffluent',
                                                                   'ethnicity_asian', 'ethnicity_other', 'ethnicity_unknown', 'ethnicity_white',
                                                                   'smoking_ex_smoker', 'smoking_never_smoked', 'smoking_smoker', 'smoking_unknown',
                                                                   'season_autumn', 'season_spring', 'season_summer', 'season_winter',
                                                                   'period_during_pandemic', 'period_post_2nd_lockdown', 'period_prepandemic',
                                                                   'ab_binary', 'antibacterial_brit'])

df_final_tab = df_concated.sort_values("variable").reset_index(drop=True)
# df_final_tab

In [7]:
#select relevant columns of each df
summary_table_cases_lrti_incdt = summary_table_cases_lrti_incdt[['variable', 'count']]
summary_table_cases_lrti_prevl = summary_table_cases_lrti_prevl[['variable', 'count']]
summary_table_cases_urti_combined_incdt = summary_table_cases_urti_combined_incdt[['variable', 'count']]
summary_table_cases_urti_combined_prevl = summary_table_cases_urti_combined_prevl[['variable', 'count']]
summary_table_cases_uti_incdt = summary_table_cases_uti_incdt[['variable', 'count']]
summary_table_cases_uti_prevl = summary_table_cases_uti_prevl[['variable', 'count']]
summary_table_cases_sinusitis_incdt = summary_table_cases_sinusitis_incdt[['variable', 'count']]
summary_table_cases_sinusitis_prevl = summary_table_cases_sinusitis_prevl[['variable', 'count']]
summary_table_cases_otmedia_incdt = summary_table_cases_otmedia_incdt[['variable', 'count']]
summary_table_cases_otmedia_prevl = summary_table_cases_otmedia_prevl[['variable', 'count']]
summary_table_cases_ot_externa_incdt = summary_table_cases_ot_externa_incdt[['variable', 'count']]
summary_table_cases_ot_externa_prevl = summary_table_cases_ot_externa_prevl[['variable', 'count']]
summary_table_cases_urti_incdt = summary_table_cases_urti_incdt[['variable', 'count']]
summary_table_cases_urti_prevl = summary_table_cases_urti_prevl[['variable', 'count']]
summary_table_cases_cough_incdt = summary_table_cases_cough_incdt[['variable', 'count']]
summary_table_cases_cough_prevl = summary_table_cases_cough_prevl[['variable', 'count']]
summary_table_cases_cough_cold_incdt = summary_table_cases_cough_cold_incdt[['variable', 'count']]
summary_table_cases_cough_cold_prevl = summary_table_cases_cough_cold_prevl[['variable', 'count']]
summary_table_cases_throat_incdt = summary_table_cases_throat_incdt[['variable', 'count']]
summary_table_cases_throat_prevl = summary_table_cases_throat_prevl[['variable', 'count']]

In [8]:
#merge hosped dfs
dfs_tab_cases = [summary_table_cases_lrti_incdt, summary_table_cases_lrti_prevl,  
                summary_table_cases_urti_combined_incdt, summary_table_cases_urti_combined_prevl, 
                summary_table_cases_uti_incdt, summary_table_cases_uti_prevl, 
                summary_table_cases_sinusitis_incdt, summary_table_cases_sinusitis_prevl, 
                summary_table_cases_otmedia_incdt, summary_table_cases_otmedia_prevl, 
                summary_table_cases_ot_externa_incdt, summary_table_cases_ot_externa_prevl, 
                summary_table_cases_urti_incdt, summary_table_cases_urti_prevl, 
                summary_table_cases_cough_incdt, summary_table_cases_cough_prevl, 
                summary_table_cases_cough_cold_incdt, summary_table_cases_cough_cold_prevl, 
                summary_table_cases_throat_incdt, summary_table_cases_throat_prevl, 
                ]

#replace any column that contains ab_date with ab_binray
for i in dfs_tab_cases:
    i.loc[i['variable'].str.contains('_ab_date'), 'variable'] = 'ab_binary'

#concat dfs and assign column names
df_concated_cases = pd.concat([i.set_index('variable') for i in dfs_tab],axis=1, join='outer').reset_index()
df_concated_cases.columns = ['variable', 
                        'lrti_incdt', 'lrti_prevl', 
                        'urti_combined_incdt', 'urti_combined_prevl',
                        'uti_incdt', 'uti_prevl',
                        'sinusitis_incdt', 'sinusitis_prevl',
                        'otmedia_incdt', 'otmedia_prevl',
                        'ot_externa_incdt', 'ot_externa_prevl',
                        'urti_incdt', 'urti_prevl',
                        'cough_incdt', 'cough_prevl',
                        'cough_cold_incdt', 'cough_cold_prevl',
                        'throat_incdt', 'throat_prevl'
                        ]

#custom sort factors of variables column 
df_concated_cases['variable'] = pd.Categorical(df_concated_cases['variable'], ['age_cat_15_24', 'age_cat_25_34', 'age_cat_35_44', 'age_cat_45_54', 'age_cat_55_64', 'age_cat_65_74', 'age_cat_75_more',
                                                                                'sex_female', 'sex_male',
                                                                                'CCI_cat_high', 'CCI_cat_low', 'CCI_cat_medium', 'CCI_cat_very_high', 'CCI_cat_very_low',
                                                                                'flu_vaccine_no', 'flu_vaccine_yes',
                                                                                'bmi_cat_healthy_weight', 'bmi_cat_obese', 'bmi_cat_overweight', 'bmi_cat_underweight', 'bmi_cat_unknown',
                                                                                'region_east', 'region_east_midlands', 'region_london', 'region_north_east', 'region_north_west', 'region_south_east', 'region_south_west', 'region_west_midlands', 'region_yorkshire',
                                                                                'imd_affluent', 'imd_medium', 'imd_unaffluent', 'imd_unknown', 'imd_very_affluent', 'imd_very_unaffluent',
                                                                                'ethnicity_asian', 'ethnicity_other', 'ethnicity_unknown', 'ethnicity_white',
                                                                                'smoking_ex_smoker', 'smoking_never_smoked', 'smoking_smoker', 'smoking_unknown',
                                                                                'season_autumn', 'season_spring', 'season_summer', 'season_winter',
                                                                                'period_during_pandemic', 'period_post_2nd_lockdown', 'period_prepandemic',
                                                                                'ab_binary', 'antibacterial_brit'])
df_final_tab_cases = df_concated_cases.sort_values("variable").reset_index(drop=True)
# df_final_tab_cases

## 1- process data to make a table of counts and rates, and save it

In [11]:
data_hosped = data_process(df_final_tab, df_final_tab_cases, 'lrti_incdt')
# data_hosped = data_process(df_final_tab, data_hosped, 'lrti_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'lrti_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_combined_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_combined_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'uti_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'uti_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'sinusitis_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'sinusitis_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'otmedia_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'otmedia_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'ot_externa_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'ot_externa_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_cold_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_cold_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'throat_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'throat_prevl')

#save hosped count and rate table
data_hosped.to_csv(f'../output/aggregate_cases_table/cases_count_rate_table_all_infections.csv', index=False)
data_hosped

,variable,lrti_incdt,lrti_prevl,urti_combined_incdt,urti_combined_prevl,uti_incdt,uti_prevl,sinusitis_incdt,sinusitis_prevl,otmedia_incdt,...,ot_externa_incdt,ot_externa_prevl,urti_incdt,urti_prevl,cough_incdt,cough_prevl,cough_cold_incdt,cough_cold_prevl,throat_incdt,throat_prevl
0,age_cat_15_24,"23,070 (1000.0)","2,570 (1000.0)","90,745 (1000.0)","10,120 (1000.0)","23,030 (1000.0)","2,560 (1000.0)","22,640 (1000.0)","2,580 (1000.0)","23,095 (1000.0)",...,"23,405 (1000.0)","2,625 (1000.0)","22,635 (1000.0)","2,560 (1000.0)","22,675 (1000.0)","2,530 (1000.0)","22,905 (1000.0)","2,545 (1000.0)","22,530 (1000.0)","2,485 (1000.0)"
1,age_cat_25_34,"22,880 (1000.0)","2,545 (1000.0)","92,285 (1000.0)","10,265 (1000.0)","22,860 (1000.0)","2,580 (1000.0)","22,845 (1000.0)","2,530 (1000.0)","23,060 (1000.0)",...,"22,640 (1000.0)","2,510 (1000.0)","23,050 (1000.0)","2,610 (1000.0)","23,075 (1000.0)","2,635 (1000.0)","23,230 (1000.0)","2,595 (1000.0)","22,930 (1000.0)","2,430 (1000.0)"
2,age_cat_35_44,"23,080 (1000.0)","2,525 (1000.0)","90,455 (1000.0)","10,185 (1000.0)","23,275 (1000.0)","2,505 (1000.0)","22,985 (1000.0)","2,700 (1000.0)","23,260 (1000.0)",...,"22,910 (1000.0)","2,530 (1000.0)","22,555 (1000.0)","2,565 (1000.0)","22,560 (1000.0)","2,590 (1000.0)","22,875 (1000.0)","2,525 (1000.0)","22,465 (1000.0)","2,505 (1000.0)"
3,age_cat_45_54,"22,990 (1000.0)","2,610 (1000.0)","91,475 (1000.0)","10,030 (1000.0)","23,190 (1000.0)","2,610 (1000.0)","22,895 (1000.0)","2,555 (1000.0)","23,280 (1000.0)",...,"22,870 (1000.0)","2,530 (1000.0)","22,790 (1000.0)","2,540 (1000.0)","22,670 (1000.0)","2,495 (1000.0)","23,110 (1000.0)","2,515 (1000.0)","22,905 (1000.0)","2,480 (1000.0)"
4,age_cat_55_64,"23,495 (1000.0)","2,635 (1000.0)","91,355 (1000.0)","10,210 (1000.0)","22,570 (1000.0)","2,590 (1000.0)","23,150 (1000.0)","2,645 (1000.0)","23,310 (1000.0)",...,"22,925 (1000.0)","2,505 (1000.0)","22,720 (1000.0)","2,575 (1000.0)","22,835 (1000.0)","2,500 (1000.0)","23,015 (1000.0)","2,610 (1000.0)","22,780 (1000.0)","2,525 (1000.0)"
5,age_cat_65_74,"23,055 (1000.0)","2,680 (1000.0)","91,875 (1000.0)","10,255 (1000.0)","23,175 (1000.0)","2,560 (1000.0)","23,020 (1000.0)","2,505 (1000.0)","22,825 (1000.0)",...,"22,525 (1000.0)","2,555 (1000.0)","23,300 (1000.0)","2,620 (1000.0)","22,865 (1000.0)","2,540 (1000.0)","22,935 (1000.0)","2,535 (1000.0)","22,780 (1000.0)","2,560 (1000.0)"
6,age_cat_75_more,"23,050 (1000.0)","2,630 (1000.0)","90,940 (1000.0)","10,215 (1000.0)","22,910 (1000.0)","2,600 (1000.0)","22,705 (1000.0)","2,560 (1000.0)","23,220 (1000.0)",...,"23,200 (1000.0)","2,600 (1000.0)","22,265 (1000.0)","2,540 (1000.0)","23,150 (1000.0)","2,490 (1000.0)","22,585 (1000.0)","2,600 (1000.0)","22,940 (1000.0)","2,580 (1000.0)"
7,sex_female,"82,555 (1000.0)","9,135 (1000.0)","324,945 (1000.0)","36,495 (1000.0)","81,890 (1000.0)","9,205 (1000.0)","81,805 (1000.0)","9,275 (1000.0)","82,750 (1000.0)",...,"81,970 (1000.0)","9,140 (1000.0)","80,845 (1000.0)","9,220 (1000.0)","81,920 (1000.0)","9,170 (1000.0)","81,515 (1000.0)","9,155 (1000.0)","80,670 (1000.0)","8,955 (1000.0)"
8,sex_male,"79,060 (1000.0)","9,055 (1000.0)","314,180 (1000.0)","34,785 (1000.0)","79,125 (1000.0)","8,800 (1000.0)","78,430 (1000.0)","8,800 (1000.0)","79,305 (1000.0)",...,"78,505 (1000.0)","8,720 (1000.0)","78,470 (1000.0)","8,790 (1000.0)","77,910 (1000.0)","8,615 (1000.0)","79,140 (1000.0)","8,765 (1000.0)","78,660 (1000.0)","8,610 (1000.0)"
9,CCI_cat_high,"16,920 (1000.0)","1,875 (1000.0)","67,535 (1000.0)","7,415 (1000.0)","17,010 (1000.0)","1,960 (1000.0)","16,630 (1000.0)","1,855 (1000.0)","16,695 (1000.0)",...,"16,930 (1000.0)","1,870 (1000.0)","16,745 (1000.0)","1,885 (1000.0)","16,695 (1000.0)","1,845 (1000.0)","17,100 (1000.0)","1,880 (1000.0)","16,995 (1000.0)","1,805 (1000.0)"
